# Using qucumber's *Observable* module

This tutorial will guide the user through how to use the built-in *Observable* module in QuCumber. Essentially, any user-made classes that are inherited from *Observable* will only need to contain a function required in order to calculate that observable from a set of data points.

## Train an RBM

Firstly, an RBM needs to be trained. If you are unsure how to do this, please refer to our tutorials on training an RBM. An RBM with a positive-real wavefunction on a transverse-field Ising model (TFIM) with 10 sites. The training data has been generated and is contained in the file *../01_Ising/tfim1d_N10_train_samples.txt*.  It contains 10,000 measurements of the $S^z$ states of 10 qubits, represented as zeros or ones.

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

from qucumber.nn_states import PositiveWavefunction
from qucumber.callbacks import MetricEvaluator
import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data

from qucumber.observables import Observable

train_samples_path = '../01_Ising/tfim1d_train.txt'
psi_path           = '../01_Ising/tfim1d_psi.txt'

train_samples,target_psi = data.load_data(train_samples_path,psi_path)

nv = train_samples.shape[-1]
nh = nv

nn_state = PositiveWavefunction.autoload("../01_Ising/saved_weights.pt")

## Calculate an observable

With our RBM trained and the parameters of the machine stored within *nn_state*, let's now define our example observable called *PIQuIL*. The *PIQuIL* observable takes an $S^z$ measurement at a site and multiplies it by the measurement two sites from it. There is also a parameter, P, that determines the strength of each of these interactions. For example, for the dataset $(-1,1,1,-1), (1,1,1,1)$ and $(1,1,-1,1)$ with P = 2, the *PIQuIL* for each data point would be $\left( 2(-1\times1) + 2(1\times-1) = -4 \right), \left( 2(1\times1) + 2(1\times1) = 4 \right)$ and $\left( 2(1\times-1) + 2(1\times1) = 0 \right)$, respectively.

To calculate PIQuIL, we need a function that take zeros and ones and converts them to -1 and 1, respectively.

In [2]:
def to_pm1(samples):
    samples.mul(2.).sub(1.)

Now let's define the *PIQuIL* class (inherited from *Observables*). The *Observable* module will do all of the required sampling for us. We just have to supply any class that inherits from *Observable* with the any other required parameters (P in our case).

In [3]:
class PIQuIL(Observable):
    
    def __init__(self, P, num_samples):
        super(PIQuIL, self).__init__()
        self.P = P
        self.num_samples = num_samples # required argument for sampling
    
    # function that calculates the PIQuIL
    def interaction(self, samples):
        to_pm1(samples)
        interaction_ = 0
        for i in range(samples.shape[-1]):
            if (i+3) > samples.shape[-1]:
                continue
            else:
                interaction_ += (self.P*samples[:,i]*samples[:,i+2])
    
        return interaction_
    
    # Required
    def apply(self, nn_state, samples):
        return self.interaction(samples)

The *apply* function is contained in the *Observable* module, but is overwritten here. The overwritten function will give the *statistics* function in *Observable* a torch tensor object containing the observable's value for each data point. The *statistics* function will then calculate the mean, standard error and variance of your observable given all of the generated samples.

Now, let's calculate *PIQuIL*.

In [4]:
P = 1.2
num_samples = 1000

piquil = PIQuIL(P, num_samples)

piquil_stats = piquil.statistics(nn_state, num_samples)

print ('Mean PIQuIL: %.4f' % piquil_stats['mean'], '+/- %.4f' % piquil_stats['std_error'])
print ('Variance: %.4f' % piquil_stats['variance'])

Mean PIQuIL: 3.3828 +/- 0.1048
Variance: 10.9928


That's all. Here is a generic template for you to try using *Observable* yourself.

In [5]:
# import statements (most likely need torch)

from qucumber.observables import Observable #required

class YourObservable(Observable):

    def __init__(self, num_samples, your_args1):
        super(YourObservable, self).__init__()
        self.num_samples = num_samples
        self.your_args1  = your_args1
        
    def calculate_your_observable(self, generated_samples, your_args2):
    
        # insert how to calculate your observable here
        
        # return a torch tensor containing the observable for each data point
        return your_observable
        
    def apply(self, generated_samples, nn_state, your_args2): 
        # arguments of "apply" must be in that order
        return self.calculate_your_observable(generated_samples, your_args2)   